Program to convert a FreePlane mindmap to a regular document.

We are using freeplane-io module of Python to parse and manage the mindmap (XML) file.

In [73]:
"""
A program to convert Freeplane based mindmaps to corresponding PDF documents, but with style! :)
If the mindmap is prepared, following certain conventions, you can produce print-quality PDF
documents using fp-convert.

Author: K Raghu Prasad <raghuprasad AT duck.com>
Copyright: ©2024-25 K Raghu Prasad <raghuprasad AT duck.com>
Licence: Apache 2.0 Licence
"""
import os, freeplane
from pylatex import (
    Alignat,
    Axis,
    Command,
    Document,
    Figure,
    FlushLeft,
    Foot,
    Head,
    Itemize,
    Label,
    LargeText,
    LineBreak,
    Math,
    Matrix,
    MediumText,
    MiniPage,
    NewLine,
    NoEscape,
    Package,
    PageStyle,
    Plot,
    Section,
    Subsection,
    Subsubsection,
    Tabular,
    TikZ,
    simple_page_number,
)
from pylatex.utils import bold, italic
from pytablewriter import LatexTableWriter

class TableTheme:
    header_color = "Apricot",
    rowcolor_1 = "gray!25"
    rowcolor_2 = "white"
    line_color = "red"
    
class Theme:
    table = TableTheme()
    
theme = Theme()

class FPDocument(Document):
    """
    Document class to build a Freeplane Document (from a Freeplane Mindmap).
    """
    def __init__(self, geometry_options={"tmargin": "0.5in", "lmargin": "0.5in",
                                         "rmargin": "0.5in", "bmargin": "0.5in"}):
        #, title="<Missing Title>", author="<Missing Author>", date=NoEscape(r"\today")):
        super().__init__(geometry_options)
        self.apply_theme(theme)
        
        
    def generate_header(self, title="<Missing Title>", author="<Missing Auther>",
                        company="<Missing Organization", date=NoEscape(r"\today"),
                        lfooter="Left Footer", rfooter="Right Footer"):
        self.preamble.append(Command("title", title))
        self.preamble.append(Command("author", author))
        self.preamble.append(Command("date", date))
        
        # Add document header
        header = PageStyle("header")
        
        # Create left header
        with header.create(Head("L")):
            header.append("Page date: ")
            header.append(LineBreak())
            header.append("R3")
            
        # Create center header
        with header.create(Head("C")):
            header.append(company)
            
        # Create right header
        with header.create(Head("R")):
            header.append(simple_page_number())
            
        # Create left footer
        with header.create(Foot("L")):
            header.append(NoEscape(r'\tiny{Generated by \href{https://www.python.org}{fp-convert}.}'))
        # Create center footer
        with header.create(Foot("C")):
            header.append(NoEscape(r'\small{©2024-25 Example Corporation}'))
            
        # Create right footer
        with header.create(Foot("R")):
            header.append("Right Footer")

        self.preamble.append(header)
        self.change_document_style("header")

        self.append(NoEscape(r"\maketitle"))
        
        # Add Heading
        #with self.create(MiniPage(align="c")):
        #    self.append(LargeText(bold("Title")))
        #    self.append(LineBreak())
        #    self.append(MediumText(bold("As at:")))
        

    def apply_theme(self, theme):
        self.theme = theme
        self.append(NoEscape(r'\rowcolors{2}{'+theme.table.rowcolor_1+r'}{'+theme.table.rowcolor_2+r'}'))
    

Prepare some utility functions to build certain sections of the document using Freeplane Nodes.

In [78]:

def get_id(id):
    """
    Replace _ with : in the ID of the nodes created by FP.
    """
    return id.replace("_", ":")


def get_block_types(node):
    """
    Identifies the type of LaTeX block to be required for supplied list of nodes.
    Here we traverse the children of the supplied node and determine and categorize
    the required types of blocks and returns a list containing the list of nodes
    applicable for each block-type. Currently only figures and tables are handled.
    """
    print(f"Received node '{node}' for checking block type.")
    ret = list()
    for child in node.children:
        if child.imagepath:
            ret.append(("fig", child))
            if child.children:  # It is possible that there could be tabular blocks after images
                ret.extend(get_block_types(child))
        else:
            # If image is not present for a node after level 3, it is pressumed that rest
            # of its siblings too are not going to have any image, and hence, they all are
            # meant to be shown as parts of a table only.
            ret.append(("tbl", node))
            break
    return ret


def build_figure(node, doc):
    if node.imagepath:
        fig = Figure(position='!htb')
        fig.add_image(node.imagepath, width=NoEscape(r'0.7\textwidth'), placement=NoEscape(r'\centering'))
        fig.add_caption(node)
        fig.append(NoEscape(r'\label{' + get_id(node.id + '}')))
        return fig
    return "%"
        
    #with doc.create(Figure(position='!htb')) as fig:
    #    # Add an image from the local directory (use the correct image file name and path)
    #    fig.add_image(node.imagepath, width=NoEscape(r'0.7\textwidth'), placement=NoEscape(r'\centering'))
    #    # Add a caption to the image
    #    fig.add_caption(node)
    #    fig.append(NoEscape(r'\label{' + get_id(node.id + '}')))
    #return " "

def build_recursive_list(node, doc, level):
    """
    Build and return a recursive list of lists as long as child nodes are present.
    """
    if node.children:
        itmz =  Itemize()
        for child in node.children:
            content = str(child).split(":", 1)
            if len(content) == 2:
                itmz.add_item(NoEscape(f"{bold(content[0])}: {content[1]}"))
            #elif child.notes:
            #    itmz.add_item(NoEscape(f"{bold(content[0])}"))
            else:
                itmz.add_item(child)
            if child.notes:
                itmz.append(NewLine())
                itmz.append(child.notes)
            if child.children:
                itmz.append(build_recursive_list(child, doc, level+1))
        return itmz
    return ""

    
def build_table(node, doc):
    """
    Build a tabular layout using the tree of information obtained from the supplied of node.
    """
    if node.children:
        col1 = dict()

        for field in node.children:
            if field:
                print(f"Field is {field}")
                col1[str(field)] = {str.strip(str(d).split(":")[0]): str.strip(str(d).split(":")[1]) for d in field.children}
        col_hdrs = sorted(list({e for d in col1.values() for e in d.keys()}))

        tab = Tabular("l" * (1+len(col_hdrs)))
        tab.add_hline(color=doc.theme.table.line_color)
        row = list(" ")
        row.extend([bold(hdr) for hdr in col_hdrs])
        tab.add_row(*row, color=doc.theme.table.header_color, strict=True)
        tab.add_hline(color=doc.theme.table.line_color)
        for field in sorted(col1.keys()):
            row = [field,]
            for col in col_hdrs:
                row.append(col1[field].get(col, ""))
            tab.add_row(row)
        tab.add_hline(color=doc.theme.table.line_color)
        return tab
    return ""


def traverse_children(node, indent, doc):
    """
    Traverse the node-tree and build sections, subsections, and further sections and
    tables based on the depth of the node under process.
    """
    if node:
        # For the purpose of debugging only
        print("  "*indent, f"[{node.id}]", node)
        if node.imagepath:
            print("  "*indent, f"[{node.id}]", node.imagepath)
        if node.icons:
            print("  "*indent, f"[{node.id}]", node.icons)
         
        if indent == 1:
            blocks = [Section(f"{node}", label=Label(get_id(node.id))),]
        elif indent == 2:
            blocks = [Subsection(f"{node}", label=Label(get_id(node.id))),]
        elif indent == 3:
            blocks = [Subsubsection(f"{node}", label=Label(get_id(node.id))),]
            if node.children:
                if node.icons and 'links/file/generic' in node.icons:  # Table is to be built
                    blocks.append(build_table(node, doc))
                elif node.icons and 'list' in node.icons:  # List is to be built
                    blocks.append(build_recursive_list(node, doc, 1))
                elif node.icons and 'image' in node.icons:  # List is to be built
                    for child in node.children:
                        blocks.append(build_figure(child, doc))
        else:
            return
        
        with doc.create(blocks[0]):
            if node.notes:
                doc.append(node.notes)

            if len(blocks) > 1:
                with doc.create(FlushLeft()) as laligned:
                    for element in blocks[1:]:
                        laligned.append(element)
        for child in node.children:
            traverse_children(child, indent+1, doc)

Then we access the root node and start iterating the children there onwards.

In [79]:
#traverse_children(root, 1)

if __name__ == '__main__':
    mindmap_file_path = "./FPConvertTest.mm"
    mm = freeplane.Mindmap(mindmap_file_path)
    node = mm.rootnode

    geometry_options = {"margin": "0.5cm"}
    doc = FPDocument(geometry_options)
    #doc = Document(geometry_options=geometry_options)
    doc.generate_header(node, "Ravi Prakash", "Example Corporations LLP")
    doc.packages.append(Package('xcolor', options=("dvipsnames", "table")))
    doc.packages.append(Package('hyperref'))
    doc.packages.append(Package('placeins', options=("section",)))
    
    
    #with doc.create(Section("Introduction")):
    #    doc.append(f"Document retrieved from mindmap {mindmap_file_path}.\nRoot node is {node}.")
    for child in node.children:
        traverse_children(child, 1, doc)
    doc.generate_pdf(node.id, clean_tex=False)

   [ID_86526980] User Registration
     [ID_310393498] User Registration Form
       [ID_861642195] Form-Fields
       [ID_861642195] ['links/file/generic']
Field is Email Address
Field is Mobile Number
Field is Full Name
Field is Address
         [ID_1704485931] Email Address
         [ID_1429610052] Mobile Number
         [ID_1876960572] Full Name
         [ID_615179840] Address
       [ID_303877642] Buttons
       [ID_303877642] ['list']
         [ID_1572971016] Register
         [ID_1737270551] Reset
       [ID_1390704400] Links
       [ID_1390704400] ['list']
         [ID_1580478443] Login Here
       [ID_777556651] Wireframes
       [ID_777556651] ['image']
         [ID_571525042] User-registration Screen
         [ID_571525042] images/user_register.png
         [ID_1641632316] Record exists already
         [ID_1641632316] images/user_exists_already.png
     [ID_96957053] Level 1.2
       [ID_510466081] Level 1.2.1
         [ID_1292557226] Level 1.2.1.1
         [ID_564502373] L